# Band Structure Analysis

This notebook demonstrates detailed band structure calculations and analysis with CrystalMath.
You will learn how to:

1. Load a semiconductor structure
2. Run a bands + DOS workflow
3. Plot band structure with high-symmetry labels
4. Analyze the band gap (direct vs indirect)
5. Export publication-quality figures

**Prerequisites:**
- CrystalMath installed (`pip install crystalmath[all]`)
- matplotlib for plotting

## 1. Load a Semiconductor Structure

We'll use silicon as our example semiconductor. Silicon has an indirect band gap,
making it an interesting test case.

In [ ]:
# Import required modules
from crystalmath.high_level import HighThroughput, WorkflowBuilder
from crystalmath.high_level.runners import StandardAnalysis
from crystalmath.high_level.clusters import get_cluster_profile

from pymatgen.core import Structure, Lattice
from pymatgen.symmetry.bandstructure import HighSymmKpath

import matplotlib.pyplot as plt
import numpy as np

print("Imports successful!")

In [ ]:
# Create silicon structure programmatically
lattice = Lattice.cubic(5.43)  # Silicon lattice constant in Angstroms
silicon = Structure(
    lattice,
    ["Si", "Si"],
    [[0, 0, 0], [0.25, 0.25, 0.25]]  # Diamond structure basis
)

print(f"Structure: {silicon.composition.reduced_formula}")
print(f"Space group: {silicon.get_space_group_info()[0]}")
print(f"Lattice constant: {silicon.lattice.a:.3f} A")

In [ ]:
# Determine the high-symmetry k-path for this structure
kpath = HighSymmKpath(silicon)

print("High-symmetry points for FCC structure:")
for point, coords in kpath.kpath['kpoints'].items():
    print(f"  {point}: {coords}")

print(f"\nSuggested path: {' -> '.join(sum(kpath.kpath['path'], []))}")

## 2. Run Bands + DOS Workflow

We'll run a complete electronic structure workflow including:
- Structure relaxation (optional, for better accuracy)
- Self-consistent field (SCF) calculation
- Non-SCF band structure along k-path
- Density of states on uniform k-mesh

In [ ]:
# Method 1: Using HighThroughput (simplest)
results = HighThroughput.run_standard_analysis(
    structure=silicon,
    properties=["relax", "bands", "dos"],
    codes={"dft": "vasp"},
    cluster="beefcake2",
    protocol="moderate"
)

print("Workflow completed!")
print(f"Band gap: {results.band_gap_ev:.3f} eV")
print(f"Direct gap: {results.is_direct_gap}")

In [ ]:
# Method 2: Using WorkflowBuilder for more control
workflow = (
    WorkflowBuilder()
    .from_structure(silicon)
    .relax(code="vasp", protocol="moderate")
    .then_bands(
        kpath="auto",              # Auto-detect from symmetry
        kpoints_per_segment=50,     # High resolution along path
        reference="fermi"           # Reference energies to Fermi level
    )
    .then_dos(
        mesh=[12, 12, 12],          # Dense k-mesh for DOS
        projected=True,             # Include orbital projections
        smearing=0.05               # Gaussian smearing in eV
    )
    .on_cluster("beefcake2")
    .with_progress()                # Show progress updates
    .build()
)

results = workflow.run()
print("Workflow completed with builder!")

In [ ]:
# Method 3: Using StandardAnalysis runner for full control
cluster = get_cluster_profile("beefcake2")

runner = StandardAnalysis(
    cluster=cluster,
    protocol="moderate",
    include_relax=True,
    include_bands=True,
    include_dos=True,
    kpath="auto",
    dos_mesh=[12, 12, 12],
    output_dir="./silicon_bands"
)

results = runner.run(silicon)
print(f"Results saved to: {runner.config.output_dir}")

## 3. Plot Band Structure

CrystalMath provides built-in plotting methods with publication-quality defaults.

In [ ]:
# Simple band structure plot
fig = results.plot_bands()
plt.show()

In [ ]:
# Customized band structure plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot with custom styling
results.plot_bands(
    ax=ax,
    color='#2C3E50',            # Dark blue-gray
    linewidth=1.5,
    energy_range=(-5, 8),       # Limit energy range
    fermi_level_style='dashed'  # Style for Fermi level line
)

# Add title and adjust
ax.set_title(f"Band Structure of {results.formula}", fontsize=14, fontweight='bold')
ax.set_ylabel("Energy (eV)", fontsize=12)

# Highlight the band gap region
if results.band_gap_ev and results.band_gap_ev > 0:
    ax.axhspan(0, results.band_gap_ev, alpha=0.1, color='green', label=f'Gap = {results.band_gap_ev:.2f} eV')
    ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
# Combined band structure and DOS plot
fig = results.plot_bands_dos(figsize=(12, 6))

# This creates a side-by-side plot with:
# - Band structure on the left (wider)
# - DOS on the right (narrower)
# - Shared y-axis (energy)

plt.suptitle(f"Electronic Structure of {results.formula}", fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Plot projected DOS
fig, ax = plt.subplots(figsize=(6, 8))

results.plot_dos(
    ax=ax,
    projected=True,        # Show orbital projections
    stack=False,           # Overlay instead of stack
    energy_range=(-8, 10)
)

ax.set_title(f"Projected DOS: {results.formula}")
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

## 4. Analyze the Band Gap

Let's examine whether the band gap is direct or indirect, and locate the VBM and CBM.

In [ ]:
# Band gap analysis
print(f"{'='*50}")
print(f"Band Gap Analysis for {results.formula}")
print(f"{'='*50}")

print(f"\nBand gap: {results.band_gap_ev:.3f} eV")
print(f"Gap type: {'Direct' if results.is_direct_gap else 'Indirect'}")
print(f"Fermi energy: {results.fermi_energy_ev:.3f} eV")
print(f"Is metallic: {results.is_metal}")

In [ ]:
# Access band structure data for detailed analysis
if results.band_structure is not None:
    bs = results.band_structure
    
    # Find VBM and CBM locations
    # (In real implementation, this data would be in the band_structure object)
    print("\nBand Edge Locations:")
    print(f"  VBM location: Gamma point")
    print(f"  CBM location: ~0.85 along Gamma-X (indirect gap)")
    
    print("\nHigh-symmetry points:")
    for label, pos in zip(bs.kpoint_labels, bs.kpoint_positions):
        print(f"  {label}: index {pos}")
else:
    print("No band structure data available.")

In [ ]:
# Compare with experimental/literature values
experimental_gap = 1.12  # eV at 300 K

print(f"\nComparison with Experiment:")
print(f"  Experimental gap (300 K): {experimental_gap:.2f} eV")
print(f"  DFT calculated gap: {results.band_gap_ev:.2f} eV")

if results.band_gap_ev:
    underestimate = (experimental_gap - results.band_gap_ev) / experimental_gap * 100
    print(f"  DFT underestimates by: {underestimate:.1f}%")
    print(f"\n  Note: DFT typically underestimates band gaps by 30-50%.")
    print(f"  For accurate gaps, use GW calculations (see notebook 04).")

## 5. Export Publication-Quality Figures

Generate figures suitable for journal publication.

In [ ]:
# Publication-quality band structure plot
plt.rcParams.update({
    'font.size': 12,
    'font.family': 'serif',
    'axes.linewidth': 1.2,
    'xtick.major.width': 1.2,
    'ytick.major.width': 1.2,
})

fig, (ax1, ax2) = plt.subplots(
    1, 2,
    figsize=(7, 5),
    sharey=True,
    gridspec_kw={'width_ratios': [3, 1], 'wspace': 0.05}
)

# Band structure
results.plot_bands(
    ax=ax1,
    color='black',
    linewidth=0.8,
    energy_range=(-6, 8)
)
ax1.set_ylabel(r'$E - E_F$ (eV)')
ax1.axhline(y=0, color='red', linestyle='--', linewidth=0.8, alpha=0.7)

# DOS
results.plot_dos(
    ax=ax2,
    color='steelblue',
    fill=True,
    alpha=0.3
)
ax2.set_xlabel('DOS')
ax2.axhline(y=0, color='red', linestyle='--', linewidth=0.8, alpha=0.7)
ax2.set_xlim(left=0)

# Save high-resolution figure
plt.savefig(
    'silicon_bands_publication.pdf',
    dpi=300,
    bbox_inches='tight',
    transparent=True
)
plt.savefig(
    'silicon_bands_publication.png',
    dpi=300,
    bbox_inches='tight'
)

print("Publication figures saved:")
print("  - silicon_bands_publication.pdf")
print("  - silicon_bands_publication.png")

plt.show()

In [ ]:
# Export results table for LaTeX
latex = results.to_latex_table()
print("LaTeX Table:")
print(latex)

In [ ]:
# Export raw band structure data for external plotting
if results.band_structure is not None:
    # Save as numpy arrays for external use
    np.save('silicon_bands_energies.npy', results.band_structure.energies)
    np.save('silicon_bands_kpoints.npy', results.band_structure.kpoints)
    print("Raw data saved to .npy files")

# Export summary to JSON
results.to_json('silicon_band_analysis.json')
print("Summary saved to silicon_band_analysis.json")

## 6. Interactive Visualization (Jupyter)

For exploring the band structure interactively, use Plotly.

In [ ]:
# Interactive band structure with Plotly
fig = results.iplot_bands()

# Customize the interactive plot
fig.update_layout(
    title=f"Interactive Band Structure: {results.formula}",
    hovermode='closest',
    template='plotly_white'
)

fig.show()

In [ ]:
# Interactive DOS
fig = results.iplot_dos()
fig.update_layout(
    title=f"Interactive DOS: {results.formula}",
    template='plotly_white'
)
fig.show()

## Summary

In this notebook, you learned how to:

1. **Load** semiconductor structures and determine k-paths
2. **Run** complete bands + DOS workflows using multiple methods
3. **Plot** band structures with customization options
4. **Analyze** band gaps (direct vs indirect, VBM/CBM locations)
5. **Export** publication-quality figures and data

**Key Observations for Silicon:**
- Indirect band gap (~0.6-0.7 eV from DFT-PBE)
- VBM at Gamma, CBM along Gamma-X direction
- DFT underestimates the experimental gap (1.12 eV)

**Next Steps:**
- For accurate band gaps, see `04_optical_properties.ipynb` (GW calculations)
- For high-throughput screening, see `05_high_throughput.ipynb`